<a href="https://colab.research.google.com/github/fcoliveira-utfpr/chuva_probabilidade_sh/blob/main/analises_decendial_chuva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando e importando bibliotecas

In [49]:
#Instalando bibliotecas
!pip install pymannkendall -q

#Importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from datetime import timedelta
import math
from scipy import stats
from scipy.stats import (gamma, norm, ks_2samp, beta, weibull_min, expon, lognorm)
from matplotlib.ticker import FuncFormatter
from scipy.stats import ksone
from sklearn.preprocessing import MinMaxScaler
import pymannkendall as mk
from scipy.stats import linregress

##Importando e abrindo planilha



In [50]:
base_url = "https://raw.githubusercontent.com/fcoliveira-utfpr/chuva_probabilidade_sh/refs/heads/main/"
arquivo = "dataset_chuva_sh.csv"
url = base_url + arquivo
df = pd.read_csv(url)
data = df['Data']
df = df.drop(columns=['Data'])
df = df.replace({',': '.'}, regex=True)
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
df['Data'] = pd.to_datetime(data, format='%d/%m/%Y')
df

,Dia,Mês,Ano,Chuva (mm),lat,lon,alt,Data
0,1.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-01
1,2.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-02
2,3.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-03
3,4.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-04
4,5.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-05
...,...,...,...,...,...,...,...,...
17527,27.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-27
17528,28.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-28
17529,29.0,12.0,2023.0,9.8,-24.887224,-54.30623,238.0,2023-12-29
17530,30.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-30


##Agrupando os dados em decêndios

In [51]:
#Função para calcular decêndios
def calcular_decendio(data):
    # Data de referência para o primeiro decêndio
    data_referencia = datetime(year=data.year, month=1, day=1)
    # Calcula o número de dias entre a data de referência e a data fornecida
    dias = (data - data_referencia).days
    # Calcula o número do decêndio (1 a 37)
    decendio = (dias // 10) + 1
    # Calcula a data inicial e final do decêndio
    data_inicio = data_referencia + timedelta(days=(decendio - 1) * 10)
    data_fim = data_inicio + timedelta(days=9)
    # Ajusta o último decêndio do ano (caso o ano não termine exatamente no 360º dia)
    if decendio == 37:
        data_fim = datetime(year=data.year, month=12, day=31)
    return decendio, data_inicio, data_fim

# Aplica a função para calcular o decêndio e as datas de início e fim
df[['decêndio', 'Data_Inicial', 'Data_Final']] = df.apply(
    lambda row: pd.Series(calcular_decendio(row['Data'])), axis=1
)

# Cria uma coluna 'Ano' a partir da coluna 'Data'
df['Ano'] = df['Data'].dt.year

# Agrupa por 'Ano' e 'decêndio' e soma a coluna 'Chuva (mm)', mantendo as datas iniciais e finais
df_acumulado_dec = df.groupby(['Ano', 'decêndio']).agg({
    'Chuva (mm)': 'sum',
    'Data_Inicial': 'first',
    'Data_Final': 'first'
}).reset_index()

# Mostrar o DataFrame resultante
df_acumulado_dec

,Ano,decêndio,Chuva (mm),Data_Inicial,Data_Final
0,1976,1,47.1,1976-01-01,1976-01-10
1,1976,2,71.2,1976-01-11,1976-01-20
2,1976,3,102.2,1976-01-21,1976-01-30
3,1976,4,50.5,1976-01-31,1976-02-09
4,1976,5,86.2,1976-02-10,1976-02-19
...,...,...,...,...,...
1771,2023,33,99.4,2023-11-17,2023-11-26
1772,2023,34,47.5,2023-11-27,2023-12-06
1773,2023,35,63.0,2023-12-07,2023-12-16
1774,2023,36,35.8,2023-12-17,2023-12-26


##Encontrando FDP e FDA

In [52]:
# Analisando o melhor ajuste para cada decêncio
periodo_10 = list(range(1, df_acumulado_dec['decêndio'].iloc[-1] + 1))
res_10 = []
preci = []

for y in periodo_10:
    df_1 = df_acumulado_dec.loc[df_acumulado_dec['decêndio'] == y]

    chuva_dec = df_1['Chuva (mm)'].tolist()
    for i in chuva_dec:
      i + 1

    chuva_10 = chuva_dec
    # Crie um intervalo de valores para a precipitação
    x = np.linspace(0, max(chuva_10), 100)

    # Distribuição Normal
    mu, std = stats.norm.fit(chuva_10)
    pdf_normal = stats.norm.pdf(x, mu, std)
    D_normal, p_value_normal = stats.kstest(chuva_10, 'norm', args=(mu, std))
    cdf_normal = stats.norm.cdf(x, mu, std)

    # Distribuição Exponencial
    loc_exp, scale_exp = stats.expon.fit(chuva_10)
    pdf_exp = stats.expon.pdf(x, loc=loc_exp, scale=scale_exp)
    D_exp, p_value_exp = stats.kstest(chuva_10, 'expon', args=(loc_exp, scale_exp))
    cdf_exp = stats.expon.cdf(x, loc=loc_exp, scale=scale_exp)

    # Distribuição Gama
    shape_gama, loc_gama, scale_gama = stats.gamma.fit(chuva_10)
    pdf_gama = stats.gamma.pdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)
    D_gama, p_value_gama = stats.kstest(chuva_10, 'gamma', args=(shape_gama, loc_gama, scale_gama))
    cdf_gama = stats.gamma.cdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)

    # Distribuição Log-Normal
    shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_10)
    pdf_ln = stats.lognorm.pdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)
    D_ln, p_value_ln = stats.kstest(chuva_10, 'lognorm', args=(shape_ln, loc_ln, scale_ln))
    cdf_ln = stats.lognorm.cdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)

    n = len(chuva_10)
    alpha = 0.05  # Nível de significância
    D_critical = ksone.ppf(1 - alpha / 2, n)

    data = {
        "Distribuição": ["Normal", "Exponencial", "Gama", "Log-Normal"],
        "P-valor": [p_value_normal, p_value_exp, p_value_gama, p_value_ln],
        "Dsup": [D_normal, D_exp, D_gama, D_ln],
        "Dcrítico": [D_critical, D_critical, D_critical, D_critical],
        "Decêndio": [y, y, y, y],
        "Nº": [n, n, n, n],
        "Coeficientes": [
            f"mu={mu:.1f}, std={std:.1f}" if not np.isnan(mu) and not np.isnan(std) else "-",
            f"loc={loc_exp:.1f}, scale={scale_exp:.1f}" if not np.isnan(loc_exp) and not np.isnan(scale_exp) else "-",
            f"shape={shape_gama:.1f}, loc={loc_gama:.1f}, scale={scale_gama:.1f}" if not np.isnan(shape_gama) and not np.isnan(loc_gama) and not np.isnan(scale_gama) else "-",
            f"shape={shape_ln:.1f}, loc={loc_ln:.1f}, scale={scale_ln:.1f}" if not np.isnan(shape_ln) and not np.isnan(loc_ln) and not np.isnan(scale_ln) else "-",
        ]
    }
    res_10.append(pd.DataFrame(data))
    preci.append(chuva_10)
res_10
result = pd.concat(res_10)

# Determina o melhor ajuste com base no nível de significância
result["Melhor Ajuste"] = "Não siguinificativo"  # Começa com "Inadequado" e é atualizado se um ajuste for adequado

result.loc[result["P-valor"] > alpha, "Melhor Ajuste"] = result.loc[result["P-valor"] > alpha, "Distribuição"]

# Exiba o DataFrame ordenado por mês
result = result.sort_values(by='Decêndio').reset_index()
result = result[['Melhor Ajuste', 'P-valor', 'Dsup', 'Dcrítico', 'Decêndio', 'Nº','Coeficientes']]

#Ordenando pelo menor Dsup de cada mÊs
dat = []
for y in periodo_10:
    df_1 = result.loc[result['Decêndio'] == y]
    df_2 = df_1.sort_values(by='Dsup', ascending=True)
    dat.append(pd.DataFrame(df_2))
dat
df_res = pd.concat(dat)

for i in chuva_10:
  i - 1
df_res

/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6554: RuntimeWarning: overflow encountered in divide
  return np.sum((1 + np.log(shifted/scale)/shape**2)/shifted)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:1997: RuntimeWarning: overflow encountered in divide
  place(output, cond, self._pdf(*goodargs) / scale)


,Melhor Ajuste,P-valor,Dsup,Dcrítico,Decêndio,Nº,Coeficientes
1,Exponencial,7.388881e-01,0.095404,0.192208,1,48,"loc=0.0, scale=50.0"
0,Normal,4.208438e-01,0.123690,0.192208,1,48,"mu=50.0, std=43.2"
2,Não siguinificativo,1.780517e-02,0.217635,0.192208,1,48,"shape=0.6, loc=-0.0, scale=57.4"
3,Não siguinificativo,5.797337e-13,0.527187,0.192208,1,48,"shape=206.8, loc=-0.0, scale=0.0"
5,Exponencial,5.730078e-01,0.109656,0.192208,2,48,"loc=0.0, scale=52.3"
...,...,...,...,...,...,...,...
143,Não siguinificativo,2.744622e-13,0.533490,0.192208,36,48,"shape=181.0, loc=-0.0, scale=0.0"
146,Não siguinificativo,2.342436e-02,0.211186,0.192208,37,48,"mu=23.9, std=29.8"
144,Não siguinificativo,1.156445e-04,0.312500,0.192208,37,48,"loc=0.0, scale=23.9"
145,Não siguinificativo,2.557536e-06,0.367424,0.192208,37,48,"shape=0.3, loc=-0.0, scale=21.6"


In [54]:
df_grouped = df_res.groupby("Decêndio").first().reset_index()
df_grouped

,Decêndio,Melhor Ajuste,P-valor,Dsup,Dcrítico,Nº,Coeficientes
0,1,Exponencial,0.738888,0.095404,0.192208,48,"loc=0.0, scale=50.0"
1,2,Exponencial,0.573008,0.109656,0.192208,48,"loc=0.0, scale=52.3"
2,3,Exponencial,0.753027,0.094156,0.192208,48,"loc=0.0, scale=57.7"
3,4,Exponencial,0.441170,0.121687,0.192208,48,"loc=0.0, scale=57.0"
4,5,Exponencial,0.585680,0.108558,0.192208,48,"loc=0.0, scale=59.1"
5,6,Exponencial,0.722829,0.096806,0.192208,48,"loc=0.0, scale=57.8"
6,7,Normal,0.220013,0.148124,0.192208,48,"mu=36.0, std=34.5"
7,8,Normal,0.109150,0.170432,0.192208,48,"mu=42.7, std=44.8"
8,9,Normal,0.071907,0.182401,0.192208,48,"mu=39.7, std=43.8"
9,10,Normal,0.100204,0.172953,0.192208,48,"mu=41.3, std=43.9"


##Determiando precipitação provável


In [53]:
#Determinação da precipitação provável de cada mês
periodo_10 = list(range(1, df_acumulado_dec['decêndio'].iloc[-1] + 1))
nivel_probabilidade = [0.9, 0.8, 0.75, 0.5, 0.25, 0.20, 0.10]

resultados = []

for probabilidade in nivel_probabilidade:
    precipitacao_provavel = []

    for y in periodo_10:
      df_1 = df_acumulado_dec.loc[df_acumulado_dec['decêndio'] == y]

      chuva_dec = df_1['Chuva (mm)'].tolist()
      for i in chuva_dec:
        i + 1
      chuva_10 = chuva_dec

      df_2 = df_res.loc[df_res['Decêndio'] == y].reset_index()
      best_ajuste = df_2['Melhor Ajuste'].loc[0]
      x = np.linspace(0, max(chuva_10), 1000)
      if best_ajuste == "Normal":
        mu, std = stats.norm.fit(chuva_10)
        precipitacao_provavel_mensal = max(0, stats.norm.ppf(probabilidade, mu, std))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Exponencial":
        loc_exp, scale_exp = stats.expon.fit(chuva_10)
        precipitacao_provavel_mensal = max(0, stats.expon.ppf(probabilidade, loc_exp, scale_exp))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Gama":
        shape_gama, loc_gama, scale_gama = stats.gamma.fit(chuva_10)
        precipitacao_provavel_mensal = max(0, stats.gamma.ppf(probabilidade, shape_gama, loc_gama, scale_gama))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Log-Normal":
        shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_10)
        precipitacao_provavel_mensal = max(0, stats.lognorm.ppf(probabilidade, shape_ln, loc_ln, scale_ln))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      else:
        mu, std = stats.norm.fit(chuva_10)
        precipitacao_provavel_mensal = max(0, stats.norm.ppf(probabilidade, mu, std)) #para os que não ajustaram, foi usado Normal
        precipitacao_provavel.append(precipitacao_provavel_mensal)
    resultados.append(precipitacao_provavel)
    # Cria um DataFrame com os resultados
colunas = ["10%", "20%", "25%", "50%", "75%", "80%","90%"]
dec_num = periodo_10
df_resultados = pd.DataFrame(resultados, index=colunas, columns=dec_num)
df_prov = df_resultados.T
df_prov.rename(columns={"index": "Decêndio"}, inplace=True)

def subtract_one_if_greater_than_zero(value):
    return max(value - 1, 0)  # Garante que o valor nunca seja negativo

# Aplica a função a cada elemento do DataFrame
df_prov = df_prov.applymap(subtract_one_if_greater_than_zero)
df_prov_10 = df_prov
df_prov_10

<ipython-input-53-daab0b860af2>:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_prov = df_prov.applymap(subtract_one_if_greater_than_zero)


,10%,20%,25%,50%,75%,80%,90%
1,114.018922,79.394777,68.248291,33.624146,13.370319,10.146485,4.262977
2,119.425200,83.173603,71.503195,35.251598,14.045772,10.670408,4.510355
3,131.863957,91.867921,78.992073,38.996036,15.599855,11.875848,5.079521
4,130.156206,90.674254,77.963904,38.481952,15.386491,11.710350,5.001379
5,135.025214,94.077545,80.895339,39.947670,15.994818,12.182205,5.224172
6,132.142186,92.062394,79.159583,39.079792,15.634617,11.902811,5.092252
7,79.174419,64.010963,58.250299,35.002083,11.753868,5.993204,0.000000
8,99.094878,79.383813,71.895492,41.675000,11.454508,3.966187,0.000000
9,94.870814,75.592818,68.269022,38.712500,9.155978,1.832182,0.000000
10,96.546838,77.252042,69.921863,40.339583,10.757303,3.427125,0.000000
